# NLLB Fine-tune

documentation - https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments

# Data preprocess


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import json

# Step 1: Read the CSV file into a DataFrame
csv_file = 'url'
df = pd.read_csv(csv_file, delimiter=',')
df = df[df['altlabel_fr'].str.lower() != 'none']

# Step 3: Split the DataFrame into training (60%), validation (20%), and test (20%) sets
# Split the data into train (60%) and temp (40%)
train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)

# Split the temp data into validation (20%) and test (20%)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [ ]:
#Step 3: Create a list of dictionaries from the cleaned DataFrame
train_data = train_df[['pref_label', 'altlabel_fr']].rename(columns={'pref_label': 'source', 'altlabel_fr': 'target'}).to_dict(orient='records')
test_data = test_df[['pref_label', 'altlabel_fr']].rename(columns={'pref_label': 'source', 'altlabel_fr': 'target'}).to_dict(orient='records')

# Step 4: Convert the list to JSON format
json_data = json.dumps(train_data, indent=4, ensure_ascii=False)
# Optionally, save to a file
with open('train_data.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_data)

In [ ]:
import os
from datasets import load_dataset, DatasetDict

# Save the JSON data to a file
with open('data.json', 'w') as f:
    f.write(data)

# Define a function to load the raw dataset
def load_raw_dataset() -> DatasetDict:
    """
    Loads the dataset from a local JSON file and wraps it in a DatasetDict.

    :return: DatasetDict containing the dataset
    """
    train_dataset = list(load_dataset('json', data_files= 'data.json' ).values())[0]
    return DatasetDict({"train": train_dataset})

# Load the raw dataset
dataset = load_raw_dataset()


In [ ]:
print(type(test_data))

<class 'list'>


# Model Setup

**Language codes NLLB**
* eng_Latn: English
* deu_Latn: German
* fra_Latn: French
* ita_Latn: Italian
* nld_Latn: Dutch
* spa_Latn: Spanish
* zho_Hans: Chinese (Simplified)




In [ ]:
import torch
import sentencepiece
from transformers import AutoModelForSeq2SeqLM, NllbTokenizer

# Load the pre-trained model and tokenizer
model_name = "facebook/nllb-200-distilled-600M"  # Multiple to English translation model

tokenizer = NllbTokenizer.from_pretrained(model_name,
                                          src_lang="eng_Latn",
                                          tgt_lang="fra_Latn"
                                          )
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map='auto',
    use_cache=False
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

DATA_CONFIG will contain a data_config.json file which has,
{
  "data_prefix": "/content/sample_data",
  "train_data": "train_data.json",
  "test_data": "test_data.json",
  "lang_pairs": "eng_Latn-deu_Latn"
}

# Seq2Seq Finetuning

In [ ]:
import torch

cuda_available = torch.cuda.is_available()

print(f"Torch version: {torch.__version__}\nCuda is available: {cuda_available}")
assert cuda_available

Torch version: 2.4.0+cu121
Cuda is available: True


In [ ]:
def tokenize_function(example):
    return tokenizer(example['source'], text_target=example['target'], truncation=True)

#Tokenize the datasets
tokenized_datasets = example.map(tokenize_function, batched=True)

In [ ]:
# Define the tokenizer function
def tokenize_function(example):
    return tokenizer(example['source'], text_target=example['target'], truncation=True)

# Tokenize each example in the train_data list
train_tokenized_data = [tokenize_function(example) for example in train_data]
test_tokenized_data = [tokenize_function(example) for example in test_data]

for tokenized_example in tokenized_data:
    print(tokenized_example)

In [ ]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

output_dir = '/content/seq2seq_output'

args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=False,
    seed=1,
    num_train_epochs=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',  # Directory for storing logs
    logging_strategy="epoch",  # Log at the end of each epoch
    logging_steps=10,  # Log every 10 steps (adjust as needed)
    report_to="all"  # Report to all available loggers (TensorBoard, etc.)
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
trainer = Seq2SeqTrainer(
    model,
    args,
    tokenizer=tokenizer,
    train_dataset=train_tokenized_data,
    eval_dataset=train_tokenized_data,
    data_collator=data_collator,
    #compute_metrics=compute_metrics
    )

#trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.14 GiB. GPU 0 has a total capacity of 14.75 GiB of which 1.94 GiB is free. Process 2203 has 12.81 GiB memory in use. Of the allocated memory 12.49 GiB is allocated by PyTorch, and 187.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

**Save and Load trained Model**

In [ ]:
import os
from transformers import AutoModelForSeq2SeqLM, NllbTokenizer

# Define the directory and model save path
directory = '/path/to/your/local/directory/MT-approach/trained_model'
file_name = 'nllb_ft_en_fr.h5' #nllb_ft_src_tgt
model_save_path = os.path.join(directory, file_name)

# Ensure the directory exists
os.makedirs(directory, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(directory)
tokenizer.save_pretrained(directory)

# Load the model and tokenizer
loaded_model = AutoModelForSeq2SeqLM.from_pretrained(directory)
loaded_tokenizer = NllbTokenizer.from_pretrained(directory)

# Example usage to verify
print("Model and tokenizer saved and loaded successfully.")

In [ ]:
import os
from transformers import AutoModelForSeq2SeqLM, NllbTokenizer

# Define the directory and model save path
#path = /content/drive/MyDrive/Colab Notebooks/Thesis notebooks/MT-approach/trained_model
directory = '/content/drive/MyDrive/Colab Notebooks/Thesis notebooks/MT-approach/trained_model'
file_name = 'nllb_finetuned_fr_en.h5'
model_save_path = os.path.join(directory, file_name)

# Save the model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

# Load the model and tokenizer
loaded_model = AutoModelForSeq2SeqLM.from_pretrained(model_save_path)
loaded_tokenizer = NllbTokenizer.from_pretrained(model_save_path)

In [ ]:
def translate(label):
  input_ids = tokenizer.encode(label, return_tensors="pt").to(model.device)

  # Translate the current word to English
  translated_ids = model.generate(input_ids, forced_bos_token_id=tokenizer.encode("eng_Latn")[0], max_length=50, num_beams=4, early_stopping=True)
  # solution: tokenizer.encode("deu_Latn")[0] - AttributeError: 'NllbTokenizerFast' object has no attribute 'lang_code_to_id' - link : https://github.com/huggingface/transformers/issues/31348
  #("deu_Latn")[1]
  translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)

  return translated_text

In [ ]:
translate('Diethyl-cadmium') #SRC = "eng_Latn" TGT = "deu_Latn"